In [19]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import torch.nn.functional as F
import torch
from torch.utils.data import DataLoader, Dataset
from models import SEQUENCE_LENGTH, N_FEATURES, HIDDEN_SIZE, NUM_LAYERS, NUM_HEADS
from models import TransformerModel
from models import TransformerModelChatGPT
from models import TransformerModelClaudeOptimal
from models import TransformerModelCopilotOptimal
from models import TransformerModelCursorOptimal
from models import TransformerModelDeepseek
from models import TransformerModelDeepseekOptimal
from models import TransformerModelGrok
from models import TransformerModelGrokOptimal

import warnings
warnings.filterwarnings("ignore")

BATCH_SIZE = 32

class ActivityDataset(Dataset):
    def __init__(self, sequences, labels):
        self.sequences = sequences
        self.labels = labels
    
    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, idx):
        return self.sequences[idx], self.labels[idx]

def prepare_sequences(data, sequence_length):
    sequences = []
    labels = []
    
    for i in range(0, len(data) - sequence_length, sequence_length//2):  # 50% overlap
        seq = data[i:i + sequence_length]
        if len(seq) == sequence_length:
            sequences.append(seq)
            labels.append(seq['ActivityLabel'].mode()[0])  # Most common label in sequence
    
    return np.array(sequences), np.array(labels)

df = pd.read_csv('data\\combined\\merged_users_corrected.csv')
    
# Prepare features and labels
features = ['AccelX', 'AccelY', 'AccelZ', 'GyroX', 'GyroY', 'GyroZ']
X = df[features]
y = df['ActivityLabel']

# Scale features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=features)

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Create sequences
X_scaled['ActivityLabel'] = y_encoded
sequences, labels = prepare_sequences(X_scaled, SEQUENCE_LENGTH)

# Remove ActivityLabel column from sequences
sequences = sequences[:, :, :-1].astype(np.float32)

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    sequences, labels, test_size=0.2, random_state=42
)

# Create datasets and dataloaders
train_dataset = ActivityDataset(torch.FloatTensor(X_train), torch.LongTensor(y_train))
test_dataset = ActivityDataset(torch.FloatTensor(X_test), torch.LongTensor(y_test))

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [20]:
def load_model(model_index, model_path, num_features, num_classes, device):
    list_models = [
        TransformerModel,
        TransformerModelDeepseek,
        TransformerModelChatGPT,
        TransformerModelGrok,
        TransformerModelGrokOptimal,
        TransformerModelClaudeOptimal,
        TransformerModelDeepseekOptimal,
        TransformerModelCursorOptimal,
        TransformerModelCopilotOptimal
    ]
    model = list_models[model_index](
        input_size=num_features,
        hidden_size=64,
        num_layers=3,
        num_heads=4,
        num_classes=num_classes
    ).to(device)

    checkpoint = torch.load(model_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()
    return model

# Hàm đánh giá mô hình
def evaluate_model(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0
    
    y_true = []
    y_pred = []
    
    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)
            outputs = model(x)
            loss = criterion(outputs, y)
            total_loss += loss.item() * x.size(0)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == y).sum().item()
            total += y.size(0)
            
            y_true.extend(y.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())
    
    avg_loss = total_loss / total
    acc = correct / total
    f1 = f1_score(y_true, y_pred, average='weighted')
    return avg_loss, acc, f1

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Tiêu chí đánh giá
criterion = torch.nn.CrossEntropyLoss()

# Danh sách các mô hình cần so sánh
model_paths = [
    "results\\MinMaxScaler\\2\\TransformerModel_w100.pth", 
    "results\\MinMaxScaler\\2\\TransformerModelDeepseek_w100.pth", 
    "results\\MinMaxScaler\\2\\TransformerModelChatGPT_w100.pth", 
    "results\\MinMaxScaler\\2\\TransformerModelGrok_w100.pth", 
    'results\\MinMaxScaler\\2\\TransformerModelGrokOptimal_w100.pth', 
    'results\\MinMaxScaler\\2\\TransformerModelClaudeOptimal_w100.pth', 
    'results\\MinMaxScaler\\2\\TransformerModelDeepseekOptimal_w100.pth',
    'results\\MinMaxScaler\\2\\TransformerModelCursorOptimal_w100.pth',
    'results\\MinMaxScaler\\2\\TransformerModelCopilotOptimal_w100.pth'
]
results = {}

# Đánh giá từng mô hình
for index in range(len(model_paths)):
    model = load_model(index, model_paths[index], num_features=6, num_classes=6, device=device)
    test_loss, test_acc, test_f1 = evaluate_model(model, test_loader, criterion, device)
    results[index] = {"Loss": test_loss, "Accuracy": test_acc, "F1": test_f1}
    print(f"{index} - Loss: {test_loss:.4f} | Accuracy: {test_acc:.4f} | F1-Score: {test_f1:.4f}")

best_model_acc = max(results, key=lambda x: results[x]["Accuracy"])
best_model_f1 = max(results, key=lambda x: results[x]["F1"])
print(f"\nModel Acc tốt nhất: {best_model_acc} với Accuracy: {results[best_model_acc]['Accuracy']:.4f}")
print(f"\nModel F1 tốt nhất: {best_model_f1} với F1: {results[best_model_f1]['F1']:.4f}")

0 - Loss: 9.2468 | Accuracy: 0.2686 | F1-Score: 0.1213
1 - Loss: 0.1725 | Accuracy: 0.9675 | F1-Score: 0.9674
2 - Loss: 1.7556 | Accuracy: 0.6684 | F1-Score: 0.6674
3 - Loss: 0.4366 | Accuracy: 0.9203 | F1-Score: 0.9195
4 - Loss: 0.2964 | Accuracy: 0.9465 | F1-Score: 0.9467
5 - Loss: 0.6170 | Accuracy: 0.9171 | F1-Score: 0.9169
6 - Loss: 0.3698 | Accuracy: 0.9328 | F1-Score: 0.9322
7 - Loss: 0.4115 | Accuracy: 0.9412 | F1-Score: 0.9392
8 - Loss: 0.4060 | Accuracy: 0.9318 | F1-Score: 0.9297

Model Acc tốt nhất: 1 với Accuracy: 0.9675

Model F1 tốt nhất: 1 với F1: 0.9674


In [21]:
# Tạo confusion matrix cho mô hình tốt nhất
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

best_model_path = model_paths[best_model]
best_model = load_model(best_model, best_model_path, num_features=6, num_classes=6, device=device)
best_model.eval()

y_true = []
y_pred = []

with torch.no_grad():
    for x, y in test_loader:
        x, y = x.to(device), y.to(device)
        outputs = best_model(x)
        _, predicted = torch.max(outputs, 1)
        y_true.extend(y.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())

# Tính confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Vẽ confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.ylabel('Nhãn thực tế')
plt.xlabel('Nhãn dự đoán')
plt.show()


NameError: name 'best_model' is not defined